# "ArviZ customization with rcParams"
> "Use ArviZ rcParams to get sensible defaults right out of the box"

- toc: true
- author: Oriol Abril
- badges: true
- categories: [arviz, customization, rcparams]

Comments are not enabled for the blog, to inquiry further about the contents of the post, ask on [ArviZ Issues](https://github.com/arviz-devs/arviz/issues) or [PyMC Discourse](https://discourse.pymc.io/)

# About
ArviZ not only builds on top of matplotlib's `rcParams` but also adds its own rcParams instance to handle specific settings. This post will only graze matplotlib's rcParams, which are already detailed in [matplotlib's docs](https://matplotlib.org/1.4.1/users/customizing.html); it will dive into specific ArviZ rcParams.

# Introduction
Paraphrasing the description on rcParams in the documentation of matplotlib:

> ArviZ uses arvizrc configuration files to customize all kinds of properties, which we call rcParams. You can control the defaults of many properties in ArviZ: data loading mode (lazy or eager), automatically showing generated plots, the default information criteria and so on.

There are several ways of modifying `arviz.rcParams` instance, each of them targeted to specific needs.

# Customizing ArviZ

## arvizrc file
To define default values on a per user or per project basis, `arvizrc` file should be used. When imported, ArviZ search for an `arvizrc` file in several locations sorted below by priority:
- `$PWD/arvizrc`
- `$ARVIZ_DATA/arvizrc`
- On Linux,
  - `$XDG_CONFIG_HOME/arviz/arvizrc` (if `$XDG_CONFIG_HOME`
  is defined)
  - or `$HOME/.config/arviz/arvizrc` (if `$XDG_CONFIG_HOME`
  is not defined)
- On other platforms,
  - `$HOME/.arviz/arvizrc` if `$HOME` is defined
    
Once one of these files is found, ArviZ stops looking and loads its configuration. If none of them are present, the values hardcoded in ArviZ codebase are used. The file used to set the default values in ArviZ can be obtained with the following command:

In [1]:
import arviz as az
print(az.rcparams.get_arviz_rcfile())

/home/oriol/.config/arviz/arvizrc


ArviZ has loaded a file used to set defaults on a per user basis. Unless I use a different rc file in the current directory or modify `rcParams` as explained above, this configuration will be automatically used every time ArviZ is imported. This can be really useful to define the favourite backend or information criterion, written once in the rc file and ArviZ automatically uses the desired values.

## Dynamic rc settings
To set default values on a per file or per project basis, `rcParams` can also be modified dynamically, either overwritting a specific key:

In [2]:
az.rcParams["data.load"] = "eager"

Note that `rcParams` is the instance to be modified, exactly like in matplotlib. 

Another option is to define a dictionary with several new defaults and update rcParams all at once. 

In [3]:
rc = {
    "data.load": "lazy",
    "plot.max_subplots": 30,
    "stats.ic_scale": "negative_log",
    "plot.matplotlib.constrained_layout": False
}
az.rcParams.update(rc)

## rc_context
Eventually, to temporarily use a different set of defaults, ArviZ also has a [`rc_context`](https://arviz-devs.github.io/arviz/generated/arviz.rc_context.html#arviz.rc_context) function. Its main difference and advantage is that it is a context manager, therefore, all code executed inside the context will use the defaults defined by `rc_context` but once we exit the context, everything goes back to normal.

In [4]:
#collapse-hide
idata = az.load_arviz_data("centered_eight")
print(az.summary(idata, var_names="theta", kind="stats"))
with az.rc_context({"data.index_origin": 1}):
    print(az.summary(idata, var_names="theta", kind="stats"))
print(az.summary(idata, var_names="theta", kind="stats"))

           mean     sd  hpd_3%  hpd_97%
theta[0]  6.026  5.782  -3.707   17.337
theta[1]  4.724  4.736  -4.039   13.999
theta[2]  3.576  5.559  -6.779   13.838
theta[3]  4.478  4.939  -5.528   13.392
theta[4]  3.064  4.642  -5.972   11.547
theta[5]  3.821  4.979  -5.507   13.232
theta[6]  6.250  5.436  -3.412   16.920
theta[7]  4.544  5.521  -5.665   15.266
           mean     sd  hpd_3%  hpd_97%
theta[1]  6.026  5.782  -3.707   17.337
theta[2]  4.724  4.736  -4.039   13.999
theta[3]  3.576  5.559  -6.779   13.838
theta[4]  4.478  4.939  -5.528   13.392
theta[5]  3.064  4.642  -5.972   11.547
theta[6]  3.821  4.979  -5.507   13.232
theta[7]  6.250  5.436  -3.412   16.920
theta[8]  4.544  5.521  -5.665   15.266
           mean     sd  hpd_3%  hpd_97%
theta[0]  6.026  5.782  -3.707   17.337
theta[1]  4.724  4.736  -4.039   13.999
theta[2]  3.576  5.559  -6.779   13.838
theta[3]  4.478  4.939  -5.528   13.392
theta[4]  3.064  4.642  -5.972   11.547
theta[5]  3.821  4.979  -5.507   13.232


# ArviZ default settings
Here are the default ArviZ settings (also available in [GitHub](https://github.com/arviz-devs/arviz/blob/master/arvizrc.template))

In [5]:
from arviz.rcparams import RcParams, defaultParams

print(RcParams([(key, default) for key, (default, _) in defaultParams.items()]))

data.http_protocol    : https
data.index_origin     : 0
data.load             : lazy
data.save_warmup      : False
plot.backend          : matplotlib
plot.bokeh.bounds_x_range: auto
plot.bokeh.bounds_y_range: auto
plot.bokeh.figure.dpi : 60
plot.bokeh.figure.height: 500
plot.bokeh.figure.width: 500
plot.bokeh.layout.order: default
plot.bokeh.layout.sizing_mode: fixed
plot.bokeh.layout.toolbar_location: above
plot.bokeh.marker     : Cross
plot.bokeh.output_backend: webgl
plot.bokeh.show       : True
plot.bokeh.tools      : reset,pan,box_zoom,wheel_zoom,lasso_select,undo,save,hover
plot.matplotlib.constrained_layout: True
plot.matplotlib.show  : False
plot.max_subplots     : 40
plot.point_estimate   : mean
stats.credible_interval: 0.94
stats.ic_scale        : log
stats.information_criterion: loo
